# Setup

In [65]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
import skfuzzy.membership as mf
import skfuzzy as fuzz
import matplotlib.pyplot as plt 

In [66]:
x_maistas = np.arange(0, 100, 1)
x_restoranas = np.arange(0, 50, 1)
x_aptarnavimas = np.arange(0, 100, 1)
y_arbatpinigiai = np.arange(0, 25, 1)
maistas_neskanus = mf.trapmf(x_maistas, [0, 0, 35, 45])
maistas_skanus = mf.trapmf(x_maistas, [30, 50, 70, 85])
maistas_tobulas = mf.trapmf(x_maistas, [75, 90, 100,100])
restoranas_blogas = mf.trapmf(x_restoranas, [0, 0, 25, 30])
restoranas_neutralus = mf.trapmf(x_restoranas, [20, 30, 35, 40])
restoranas_geras = mf.trapmf(x_restoranas, [35, 40, 50, 50])
aptarnavimas_prastas = mf.trapmf(x_aptarnavimas, [0, 0, 35, 45])
aptarnavimas_vidutinis = mf.trapmf(x_aptarnavimas, [30, 45, 55, 65])
aptarnavimas_geras = mf.trapmf(x_aptarnavimas, [55, 65, 100, 100])
arbatpinigiai_mazai = fuzz.trimf(y_arbatpinigiai, [0, 0, 13])
arbatpinigiai_normaliai = fuzz.trimf(y_arbatpinigiai, [0, 13, 25])
arbatpinigiai_daug = fuzz.trimf(y_arbatpinigiai, [13, 25, 25])

In [67]:
def Rules(mais_neskanus, mais_skanus, mais_tobulas, res_blogas, res_geras, apt_prastas, apt_vidutinis, apt_geras):
    rule1 =  apt_prastas
    rule2 =  np.fmin(apt_vidutinis, np.fmin(mais_neskanus, 1-res_geras))
    rule3 =  np.fmin(res_blogas, np.fmin(mais_skanus, apt_vidutinis))
    high_probability = np.fmax(rule1, np.fmax(rule2, rule3))
    rule4 =  np.fmin(apt_geras, np.fmin(res_blogas, 1-mais_tobulas))
    rule5 =  np.fmin(mais_tobulas, np.fmin(res_blogas,apt_vidutinis))
    rule6 =  np.fmin(mais_neskanus, np.fmin(1-res_blogas,1-apt_prastas))
    small_probability = np.fmax(rule4, np.fmax(rule5,rule6))
    rule7 =  np.fmin(mais_tobulas, np.fmin(res_blogas,apt_geras))
    rule8 =  np.fmin(1-mais_neskanus, np.fmin(1-apt_prastas,1-res_blogas))
    average_probability = np.fmax(rule7, rule8)
    arbatos_palikti_mazai = np.fmin(high_probability, arbatpinigiai_mazai)
    arbatos_palikti_daug = np.fmin(average_probability, arbatpinigiai_daug)
    arbatos_palikti_normaliai = np.fmin(small_probability, arbatpinigiai_normaliai)
    return arbatos_palikti_mazai,arbatos_palikti_daug,arbatos_palikti_normaliai

In [68]:
def implikacija(input_maistas,input_restoranas,input_aptarnavimas):
    mais_neskanus = fuzz.interp_membership(x_maistas,maistas_neskanus, input_maistas) 
    mais_skanus = fuzz.interp_membership(x_maistas,maistas_skanus, input_maistas) 
    mais_tobulas = fuzz.interp_membership(x_maistas,maistas_tobulas, input_maistas)
    res_blogas = fuzz.interp_membership(x_restoranas,restoranas_blogas, input_restoranas)
    res_neutralus = fuzz.interp_membership(x_restoranas,restoranas_neutralus, input_restoranas) 
    res_geras = fuzz.interp_membership(x_restoranas,restoranas_geras, input_restoranas)
    apt_prastas = fuzz.interp_membership(x_aptarnavimas,aptarnavimas_prastas, input_aptarnavimas)
    apt_vidutinis = fuzz.interp_membership(x_aptarnavimas,aptarnavimas_vidutinis, input_aptarnavimas) 
    apt_geras = fuzz.interp_membership(x_aptarnavimas,aptarnavimas_geras, input_aptarnavimas)
    arbatos_palikti_mazai, arbatos_palikti_daug, arbatos_palikti_normaliai = Rules(
        mais_neskanus, mais_skanus, mais_tobulas, res_blogas, res_geras, apt_prastas, apt_vidutinis, apt_geras)
    arbatpinigiai = np.zeros_like(y_arbatpinigiai)
    aggregated = np.fmax(arbatos_palikti_mazai, np.fmax(arbatos_palikti_normaliai, arbatos_palikti_daug))
    arbata_centroid = fuzz.defuzz(y_arbatpinigiai, aggregated, 'centroid') 
    arbata_bisector = fuzz.defuzz(y_arbatpinigiai, aggregated, 'bisector') 
    arbata_mom = fuzz.defuzz(y_arbatpinigiai, aggregated, 'mom') 
    arbata_aktyvavimas = fuzz.interp_membership(y_arbatpinigiai, aggregated, arbata_centroid)
    fig, ax0 = plt.subplots(figsize=(8, 3)) 
    ax0.plot(y_arbatpinigiai, arbatpinigiai_mazai, 'red', linewidth=1, linestyle='--', ) 
    ax0.plot(y_arbatpinigiai, arbatpinigiai_normaliai, 'yellow', linewidth=1, linestyle='--') 
    ax0.plot(y_arbatpinigiai, arbatpinigiai_daug, 'green', linewidth=1, linestyle='--') 
    ax0.fill_between(y_arbatpinigiai, arbatpinigiai, aggregated, facecolor='purple', alpha=0.7) 
    ax0.plot([arbata_centroid, arbata_centroid], [0, arbata_aktyvavimas], 'black', linewidth=1.5, alpha=0.9) 
    ax0.plot([arbata_bisector, arbata_bisector], [0, arbata_aktyvavimas], 'blue', linewidth=1.5, alpha=0.9) 
    ax0.plot([arbata_mom, arbata_mom], [0, 0.4], 'cyan', linewidth=1.5, alpha=0.9) 
    ax0.set_title('Defuzifikacijos rezultatai') 
    # Turn off top/right axes 
    for ax in (ax0,): ax.spines['top'].set_visible(False) 
    ax.spines['right'].set_visible(False) 
    ax.get_xaxis().tick_bottom() 
    ax.get_yaxis().tick_left()
    plt.tight_layout()

    print("Maisto kokybė: ", mais_neskanus, mais_skanus, mais_tobulas)
    print("Restorano ivertinimas: ", res_blogas, res_neutralus, res_geras)
    print("Aptarnavimo kokybe: ", apt_prastas, apt_vidutinis, apt_geras)


# Start

In [69]:
interact(implikacija,
input_maistas=widgets.IntSlider(min=0, max=99, step=1, value=50),
input_restoranas=widgets.IntSlider(min=0, max=50, step=1, value=25),
input_aptarnavimas=widgets.IntSlider(min=0, max=99, step=1, value=50));

interactive(children=(IntSlider(value=50, description='input_maistas', max=99), IntSlider(value=25, descriptio…